Let's learn how to easily profile a cuda kernel, ie measure its runtime.

**Setup**

In [1]:
import numpy as np
from fastcore.basics import tuplify
import torch
from torch.profiler import profile, record_function, ProfilerActivity
from torch.profiler import schedule as profiler_schedule
from torch import allclose

from numba import cuda
from util import to_d, to_h, array_like, cdiv

In [2]:
m,n,k = 4092,4092,4092

a = to_d(np.ones((m,k)))
b = to_d(np.ones((k,n)))
def fresh_c(): return to_d(np.empty((m,n)))
c = fresh_c()

@cuda.jit(lineinfo=True)
def naive_matmul(a,b,c):
    tid_x, tid_y = cuda.grid(2)
    m,n = c.shape
    k = a.shape[1]
    if tid_x >= m or tid_y >= n: return 
    tmp = 0
    for i in range(k):
        tmp += a[tid_x, i] * b[i, tid_y]        
    c[tid_x, tid_y] = tmp

**Warmup**

In [3]:
nthreads = (32,32)
nblocks = cdiv(c.shape, nthreads)
print(f'Using {nblocks} blocks of size {nthreads}')

for _ in range(5):
    c = fresh_c()
    naive_matmul[nblocks, nthreads](a,b,c)

Using (128, 128) blocks of size (32, 32)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/numba/core/lowering.py:112: NumbaDebugInfoWarning: Could not find source for function: <function naive_matmul at 0x7f758778c670>. Debug line information may be inaccurate.
  warnings.warn(NumbaDebugInfoWarning(msg))


In [4]:
assert allclose(torch.tensor(to_h(c)), torch.tensor(a)@torch.tensor(b))

**Profile**

In [14]:
with profile(
    activities=[ProfilerActivity.CUDA], 
    schedule=profiler_schedule(wait=0, warmup=1, active=2, repeat=1) # uncommenting this line prevents logging cuda - why? setting it to wait=1 even prevents cpu logging - why?
) as prof:
    for _ in range(5):
        c = fresh_c()
        naive_matmul[nblocks, nthreads](a,b,c)
        prof.step()

STAGE:2024-05-01 14:28:40 4970:4970 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-05-01 14:28:47 4970:4970 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-05-01 14:28:47 4970:4970 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [15]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  cudaDeviceSynchronize       100.00%        2.087s       100.00%        2.087s        2.087s       0.000us         0.00%       0.000us       0.000us             1  
                                         cuLaunchKernel         0.00%      58.000us         0.00%      58.000us      29.000us       0.000us         0.00%       0.000us       0.000us             2  
         

In [27]:
def cuda_mean_runtime(prof, kernel_name, do_print=False):
    kernels = [o for o in prof.key_averages() if kernel_name in o.key]
    names = [k.key for k in kernels]
    if len(names)==0: raise RuntimeError(f"Profiling logs have no kernel with 'f{kernel_name}' in its name")
    if len(names)>1: raise RuntimeError(f"Profiling logs have multiple kernel with 'f{kernel_name}' in its name: f{names}. Please be more precise.")
    mean_runtime = kernels[0].cuda_time / 1e3 # use ms instea of µs
    if do_print: print(f'{mean_runtime/1e3:.3f}s') # print in s
    return mean_runtime

In [28]:
cuda_mean_runtime(prof, 'matmul', do_print=True);

2.012s
